In [1]:
import os
import django
import re

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from django.core.exceptions import ObjectDoesNotExist
import time

In [2]:
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'web_restaurant.settings')
django.setup()

In [5]:
from restaurant.models import Restaurant, Review, Chef

# Chrome 드라이버 설정
options = Options()
options.add_argument("--headless")  # 브라우저 창을 띄우지 않고 실행
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# 비동기 코드에서 ORM 호출 허용
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get('https://app.catchtable.co.kr/ct/shop/neo_/review?type=DINING&sortingFilter=L')


# 페이지가 로딩될 시간을 기다림
time.sleep(3)

# 페이지 스크롤 함수
def scroll_to_bottom():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # 끝까지 스크롤
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # 스크롤 후 대기 시간

        # 새로운 높이 계산 후 비교
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # 스크롤이 더 이상 불가능할 때 종료
        last_height = new_height

# 스크롤을 끝까지 내림
scroll_to_bottom()

# <br> 태그 제거 함수
def clean_review_text(text):
    return re.sub(r'<br\s*/?>', '\n', text)  # <br>을 줄바꿈으로 변환

# 평점과 리뷰 내용 수집
reviews = []
try:
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "_10fm75h6"))
    )
    # 모든 평점과 리뷰 요소를 찾음
    ratings = driver.find_elements(By.CLASS_NAME, "_10fm75h6")
    contents = driver.find_elements(By.CLASS_NAME, "review-content")

    for rating, content in zip(ratings, contents):
        # rating 값 정수로 버림
        rating_value = float(rating.text)
        review_type = 'good' if rating_value == 5 else 'bad'
        review_text = clean_review_text(content.get_attribute("innerHTML").strip())

        review_data = {
            "rating": review_type,
            "content": review_text
        }
        reviews.append(review_data)

except Exception as e:
    print(f"오류 발생: {e}")




restaurant_name = '식당네오'

try:
    restaurant = Restaurant.objects.get(restaurant_name=restaurant_name)
except ObjectDoesNotExist:
    print(f"레스토랑 '{restaurant_name}'이(가) 존재하지 않습니다. 새로 생성합니다.")
    chef, created = Chef.objects.get_or_create(chef_name='임시쉐프', defaults={"image_url": "./최강록.jpeg"})
    restaurant = Restaurant.objects.create(restaurant_name=restaurant_name, chef=chef)
    print(f"레스토랑 '{restaurant_name}'이(가) 생성되었습니다.")

# 해당 레스토랑의 기존 리뷰 삭제
existing_reviews = Review.objects.filter(restaurant=restaurant)
if existing_reviews.exists():
    existing_reviews.delete()
    print("기존 리뷰들 삭제 완료")

# 리뷰 저장
for review in reviews:
    Review.objects.create(
        restaurant=restaurant,
        review_text=review['content'],
        review_category=review['rating']
    )
    # 평점, 리뷰 출력
    print(f"평점: {review['rating']}")
    print(f"리뷰: {review['content']}")

print("리뷰 저장 완료")

# 드라이버 종료
driver.quit()



기존 리뷰 0개가 삭제되었습니다.
평점: bad
리뷰: 다들 극찬하던곳 ㅠㅠ 
왜 우리는 먹을때마다 서로의 눈을 쳐다보았던가? 
심지어 옆자리 앉은분들도 서로 갸우뚱 하며 함께 음식을 다 먹지 않고 남긴채로 .. 어색하게 서로 웃음.. 
 
예약이 어려워서 너무 기대했건만 너무 아쉽습니다.. 
심지어 마지막 볶음밥?은 두번이나 만드시다가 갸우뚱하며 실수하신것같은데 아무런 언급없이 그냥 안주시더라구요 
그 부분만큼은 안그래도 아쉬웠는데 너무 실망스러웠습니다. 
 
아그리고 
단체방 옆에는 앉지 마세요 너무 시끄럽습니다 
제제도 안해줍니다.
평점: bad
리뷰: 남편이 최강록 셰프 팬이라 기념일에 방문했어요. 
워낙 예약하기 어려운 곳이라 그만큼 기대도 컸었는데 
기대에 충족하는 식사는 아니었습니다. 
 
일단 가게가 협소해서 외투와 짐 놓을 곳이 마땅치 않았고  
출입문과 자리의 간격이 좁아서  
저희가 방문한 날처럼 날씨가 추운 날엔 
사람이 왔다갔다할 때 바깥의 한기가 그대로 느껴졌습니다. 
 
그리고 팀당 주류 한 병을 무조건 주문해야 하던데 
사정상 추천은 못해준다 하시더군요. 
그렇게 된 이유가 있었겠지만 가격대가 설명하신 것만큼 
천차만별도 아니었고  필수로 한 병을 주문해야 하는데 
추천도 안해주셔서 좀 막막했습니다. 
아무리 설명을 자세히 써놨다한들 우리나라 사람들이 
사케의 종류나 맛에 익숙한게 아니니까요. 
 
그리고 음식은 전 서비스로 주신 떡쌈?이 
제일 입맛에 맞았고, 사전에 설명하신 것처럼 음식들이 
술안주로 최적화된 메뉴라는 느낌도 못받았습니다. 
메뉴구성이 살짝 중구난방이라는 생각이 들었고 
맛도 그냥 먹을만한 정도지 엄청 맛있거나 특별하단 느낌은 
받지 못했네요.
평점: bad
리뷰: 음식 마다 설명해 주시는데 대체적으로 간이 세고 양이 너무 많아서 전체 요리중에 반정도 먹었을 때 이미 너무 배불렀어요 2인당 최소 10만원 이상 술도 시켜야 하는데 넷이라 두병 시켰더니 더 배불렀고 환기가 안된건지 넷다 눈이 너무 아팠어요 bar 자

In [4]:
from restaurant.models import Review
print(Review)

<class 'restaurant.models.Review'>
